In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import re
import sys
import itertools
import MeCab
import matplotlib.pyplot as plt
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import collections
import itertools

import scipy as sp

In [2]:
df = pd.read_excel("特許リスト(21.12.6).xlsx",header=0) # engineは環境によってエラー出たり出なかったりだから、必要に応じて変える


In [3]:
df['詳細な説明'] = df['詳細な説明'].fillna(' ') #nanを文字列に
df['全請求項'] = df['全請求項'].fillna(' ')
df["company"] = [set(i.split(",")) for i in df['出願人・権利者名']]    #出願名をスプリットしてset型に
df['datetime'] = [dt.strptime(i.split(',')[0],  '%Y.%m.%d') for i in df['公開・公表日']]  #公開・公表日を datetime64[ns] 型（ Timestamp 型）に 、複数あるのは先頭のだけに
# df['sentence'] = df['発明の名称'] +df['要約'] + df['全請求項'] .astype("str") + df['詳細な説明']  #なぜか .astype("str") いれないとエラー
# df['sentence'] = df['発明の名称'] +df['要約'] + df['詳細な説明']  
# df['sentence'] = df['発明の名称'] +df['要約']        #sentenceの対象にする項目も必要に応じて変える
df['sentence'] = df['詳細な説明']        #sentenceの対象にする項目も必要に応じて変える


In [4]:
excepword = ["位置","部","上記","上部","上端","下記","下部","下端","両端","内部","外部","前記","複数","横","図","要約","方法","手段","対象","現場","評価" ,"可能","構成","当該","以下","以上","該当","項","各","基準","端部","請求", "特許","本","一定","同上"]  #除外する特定の名詞
exceptype= ["接尾","副詞可能","数詞","助数詞","代名詞","複数"]  #除外する名詞のタイプ


# 
def tokenize_rm_setsubi(sentence):
    """ 連名詞のリストをトークンとして返す """
#     print(".",end='') #動いてるのか不安になるからプリント
    sentence  = re.sub('\d+', '', sentence)   # 数字を削除(macabでなぜか名詞に認定された数字あったからここで)
    sentence  = text=re.sub(r'[ -/:-@\[-~]', "。", sentence)#半角記号,数字,英字
    sentence  = text=re.sub(r'[！-／：-＠［-｀｛-～、-〜”’%]+/g', "。", sentence)#全角記号
    
    
    
    wakati = MeCab.Tagger("")
    node = wakati.parseToNode(sentence)
    sequence = 2   #何連名詞以上をかえすか
    dictionary = []
    prev_seq = 0  #その前にいくつ名詞続いたか
    flag = 0  #
    while node:
        word = node.surface
        hinshi = node.feature.split(",")

        if prev_seq  : # その前に名詞が1以上つづいていたら
            if  (hinshi[0] == "名詞" ) and  (hinshi[0] not in exceptype) and  (hinshi[1] not in exceptype )and (hinshi[2] not in exceptype )and (word not in excepword) : #今回、除外名詞以外だったら、
                dictionary[-1] = dictionary[-1]+word   # 辞書の最後の単語にこの単語を結合
                prev_seq  +=1
                flag = 0
                
            elif (hinshi[1] == "接尾" ): #接尾辞だったら
                dictionary[-1] = dictionary[-1]+word   # とりあえず辞書の最後の単語にこの単語を結合して
                prev_seq  +=1
                flag = 1 #フラグを1に

            elif flag == 1: # 一個前が接尾辞で今度が名詞でなかったら、一個前の接尾辞で終わってる単語を辞書から削除
                dictionary.pop(-1)   #一個前の接尾辞で終わってる単語を辞書から削除
                prev_seq  =0
                flag = 0
            else:
                flag = 0
                if prev_seq  < sequence: 
                    dictionary.pop(-1) #一個前の単語を辞書から削除
                prev_seq = 0
        else:  # その前に名詞がつづいてなくて
            if  (hinshi[0] == "名詞" or hinshi[0] == "接頭辞")  and  hinshi[0] not in exceptype and  hinshi[1] not in exceptype and hinshi[2] not in exceptype and word not in excepword :
                dictionary.append(word)
                prev_seq += 1
                flag = 0

        node = node.next  
    return dictionary



# 単語ー文書行列の作成
docs = np.array(df['sentence'])

tfidf_vectorizer = TfidfVectorizer(
    tokenizer = tokenize_rm_setsubi, #上でつくった関数
    lowercase=True,
    max_df=3000,
#     smooth_idf = False,
    min_df=10)

tfidf_vectorizer.fit(docs)
terms= tfidf_vectorizer.get_feature_names()

count_vectrizer = CountVectorizer(
    vocabulary = terms,
    tokenizer = tokenize_rm_setsubi, #上でつくった関数
)

c_matrix = count_vectrizer.transform(docs)


/Users/WatanabeShingo/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [5]:
# np.save('c_matrix_allterm',c_matrix)
sp.sparse.save_npz('c_matrix_allterm',c_matrix)
terms_df = pd.DataFrame(terms)
terms_df.to_csv('terms_df.csv', mode='w')

In [6]:
c_matrix = sp.sparse.load_npz('c_matrix_allterm.npz')
terms_df = pd.read_csv('terms_df.csv')
terms = terms_df['0'].tolist()

In [7]:
c_df = pd.DataFrame(terms, columns=["terms"])
c_df["document_frequency"] =  np.array(c_matrix.astype(bool).sum(axis=0).reshape(-1,1))
c_df["max_term_frequency"]  = c_matrix.T.max(axis=1).toarray()
c_df["sumof_term_frequency"] = np.array(c_matrix.T.sum(axis=1))
c_df.sort_values('document_frequency', ascending=False)

,terms,document_frequency,max_term_frequency,sumof_term_frequency
12332,図示例,2813,45,9155
7513,作業効率,2766,25,6254
12280,回転自在,2645,45,8078
12340,図示省略,2637,24,5965
16415,延在,2595,68,9835
...,...,...,...,...
5772,三次元構造,10,8,34
27675,管自体,10,3,13
5773,三次元点群データ,10,28,67
27660,管理用パソコン,10,25,57


In [8]:
#単語:文書内単語頻度 の辞書の列つくって、pandasにくっつける
def makeTermsTfDict(terms, vec):
    # dic = {key: val for key, val in zip(terms, vec)  if val } #dfが1より大きいtermの辞書つくる
    dic = {key: val for key, val in zip(terms, vec)}
    dic2 = {k: v for k, v in dic.items() if v != 0}
    dic3 = dict(sorted(dic2.items(), key=lambda x:x[1],reverse=True)) #dfの値でソート

    return dic3


dic = makeTermsTfDict(terms, c_matrix.getrow(1).toarray()[0]) 

for i in range(df.shape[0]):
    df.at[i,'terms'] = str(makeTermsTfDict(terms, c_matrix.getrow(i).toarray()[0] ))



In [10]:
df.to_excel('parcedPatents1217.xlsx', sheet_name='Sheet1')

In [ ]:
# asd = pd.read_json('parcedPatents.json')

In [ ]:
# d = eval(asd.at[0,'terms'])

In [ ]:
# type(d)

dict